## Pokemon Legendary Predictor

### By: MSLacerda

## Description

Neural Network to predict if a pokemon is legendary or not using some attributes of them.

In [1]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
import pandas as pd


print(pd.__version__)

Using TensorFlow backend.


0.24.1


## Normalizing the data to put in our model

In [75]:
data = pd.read_csv('./dataset/pokemon.csv')

X = data.drop(['abilities', 'classfication', 'type1', 'type2', 'name','percentage_male','generation', 'japanese_name', 'is_legendary', 'capture_rate'], axis=1)

normalized_Y = data[['is_legendary']]

normalized_X=(X-X.min())/(X.max()-X.min())
normalized_X = normalized_X.fillna(0)
normalized_Y.loc[normalized_Y['is_legendary'] >= 1 , 'is_legendary'] = 0.99

## Training our model

In [77]:
input_dim = X.shape[1]
model = Sequential()
model.add(Dense(12, input_shape=(input_dim,), activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(normalized_X, normalized_Y, epochs=150, batch_size=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 12)                384       
_________________________________________________________________
dense_41 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 9         
Total params: 497
Trainable params: 497
Non-trainable params: 0
_________________________________________________________________
Epoch 1/150
801/801 [==============================] - 1s 1ms/step - loss: 0.6882 - acc: 0.6280
Epoch 2/150
801/801 [==============================] - 0s 281us/step - loss: 0.5720 - acc: 0.9126
Epoch 3/150
801/801 [==============================] - 0s 211us/step - loss: 0.3530 - acc: 0.9126
Epoch 4/150
801/801 [==============================] - 0s 214us/step - loss: 0.2988 - acc: 0.9126
Epoch 5/150
801/80

801/801 [==============================] - 0s 243us/step - loss: 0.0317 - acc: 0.9076
Epoch 77/150
801/801 [==============================] - 0s 196us/step - loss: 0.0312 - acc: 0.9076
Epoch 78/150
801/801 [==============================] - 0s 208us/step - loss: 0.0322 - acc: 0.9076
Epoch 79/150
801/801 [==============================] - 0s 244us/step - loss: 0.0325 - acc: 0.9076
Epoch 80/150
801/801 [==============================] - 0s 177us/step - loss: 0.0302 - acc: 0.9089
Epoch 81/150
801/801 [==============================] - 0s 185us/step - loss: 0.0304 - acc: 0.9089
Epoch 82/150
801/801 [==============================] - 0s 191us/step - loss: 0.0298 - acc: 0.9089
Epoch 83/150
801/801 [==============================] - 0s 240us/step - loss: 0.0314 - acc: 0.9089
Epoch 84/150
801/801 [==============================] - 0s 206us/step - loss: 0.0296 - acc: 0.9089
Epoch 85/150
801/801 [==============================] - 0s 223us/step - loss: 0.0274 - acc: 0.9089
Epoch 86/150
801/801 [=

In [74]:
predictions = model.predict(normalized_X)
rounded = [round(x[0]) for x in predictions]
rounded[143]

1.0